In [1]:
import pandas as pd
import random

TASK1

In [2]:
cohorts=["alpha", "beta", "gamma"]
k=random.choice(cohorts)
s = []
for i in range(1, 25):
    if(i<10):
        s.append("S00" + str(i))
    else:
        s.append("S0" + str(i))

attendance_raw = []
for i in range(24):
    attendance_raw.append({"student_id": s[i],"cohort":random.choice(cohorts),"attended_sessions":random.randint(0,6),"expected_sessions":6})
for a in attendance_raw:
    print(a)

{'student_id': 'S001', 'cohort': 'alpha', 'attended_sessions': 6, 'expected_sessions': 6}
{'student_id': 'S002', 'cohort': 'gamma', 'attended_sessions': 3, 'expected_sessions': 6}
{'student_id': 'S003', 'cohort': 'beta', 'attended_sessions': 4, 'expected_sessions': 6}
{'student_id': 'S004', 'cohort': 'alpha', 'attended_sessions': 4, 'expected_sessions': 6}
{'student_id': 'S005', 'cohort': 'gamma', 'attended_sessions': 5, 'expected_sessions': 6}
{'student_id': 'S006', 'cohort': 'beta', 'attended_sessions': 3, 'expected_sessions': 6}
{'student_id': 'S007', 'cohort': 'gamma', 'attended_sessions': 5, 'expected_sessions': 6}
{'student_id': 'S008', 'cohort': 'alpha', 'attended_sessions': 1, 'expected_sessions': 6}
{'student_id': 'S009', 'cohort': 'alpha', 'attended_sessions': 1, 'expected_sessions': 6}
{'student_id': 'S010', 'cohort': 'gamma', 'attended_sessions': 4, 'expected_sessions': 6}
{'student_id': 'S011', 'cohort': 'beta', 'attended_sessions': 2, 'expected_sessions': 6}
{'student_id'

In [3]:
attendance=pd.DataFrame(attendance_raw)
print(attendance.iloc[:5])  #first 5 rows

  student_id cohort  attended_sessions  expected_sessions
0       S001  alpha                  6                  6
1       S002  gamma                  3                  6
2       S003   beta                  4                  6
3       S004  alpha                  4                  6
4       S005  gamma                  5                  6


In [4]:
attendance.info()

<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   student_id         24 non-null     str  
 1   cohort             24 non-null     str  
 2   attended_sessions  24 non-null     int64
 3   expected_sessions  24 non-null     int64
dtypes: int64(2), str(2)
memory usage: 900.0 bytes


TASK2

In [5]:
attendance_indexed=attendance.set_index("student_id")
print(attendance_indexed)

           cohort  attended_sessions  expected_sessions
student_id                                             
S001        alpha                  6                  6
S002        gamma                  3                  6
S003         beta                  4                  6
S004        alpha                  4                  6
S005        gamma                  5                  6
S006         beta                  3                  6
S007        gamma                  5                  6
S008        alpha                  1                  6
S009        alpha                  1                  6
S010        gamma                  4                  6
S011         beta                  2                  6
S012        alpha                  6                  6
S013        gamma                  4                  6
S014        gamma                  2                  6
S015         beta                  1                  6
S016         beta                  6            

In [6]:
excused_absences = pd.Series({
    "S001": 1,
    "S003": 2,
    "S005": 1,
    "S007": 0,
    "S010": 1,
    "S015": 2,
    "S018": 1,
    "S021": 1,
    "S024": 2,
    "S030": 1,#missing
    "S031": 2#missing
})
attendance_indexed["adjusted_attendance"] = (attendance_indexed["attended_sessions"] + excused_absences)
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,6,6,7.0
S002,gamma,3,6,NaN
S003,beta,4,6,6.0
S004,alpha,4,6,NaN
S005,gamma,5,6,6.0
S006,beta,3,6,NaN
S007,gamma,5,6,5.0
S008,alpha,1,6,NaN
S009,alpha,1,6,NaN


In [7]:
attendance_indexed[attendance_indexed["adjusted_attendance"].isna()]#Confirm that rows without matching IDs become missing in adjusted_attendance

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S002,gamma,3,6,NaN
S004,alpha,4,6,NaN
S006,beta,3,6,NaN
S008,alpha,1,6,NaN
S009,alpha,1,6,NaN
S011,beta,2,6,NaN
S012,alpha,6,6,NaN
S013,gamma,4,6,NaN
S014,gamma,2,6,NaN


In [8]:
attendance_indexed["adjusted_attendance"]=attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,6,6,7.0
S002,gamma,3,6,3.0
S003,beta,4,6,6.0
S004,alpha,4,6,4.0
S005,gamma,5,6,6.0
S006,beta,3,6,3.0
S007,gamma,5,6,5.0
S008,alpha,1,6,1.0
S009,alpha,1,6,1.0


TASK3

In [9]:
attendance_indexed.loc["S006","cohort"]=" GaMma "
attendance_indexed.loc["S012","cohort"]=" gamMA"
attendance_indexed.loc["S020","cohort"]=" AlPha "
attendance_indexed.loc["S021","cohort"]=" GAMma "
attendance_indexed.loc["S023","cohort"]=" ALPHA "
attendance_indexed.loc["S024","cohort"]=" GAMMa "
print("before normalize",attendance_indexed["cohort"].unique())

before normalize <StringArray>
[  'alpha',   'gamma',    'beta', ' GaMma ',  ' gamMA', ' AlPha ', ' GAMma ',
 ' ALPHA ', ' GAMMa ']
Length: 9, dtype: str


In [11]:
attendance_indexed["cohort"] = attendance_indexed["cohort"].str.strip().str.lower()
cleaned=set(attendance_indexed["cohort"].unique())
print("after normalize",cleaned)

after normalize {'gamma', 'beta', 'alpha'}


TASK4

In [12]:
low_attendance=attendance_indexed[attendance_indexed["attended_sessions"]<attendance_indexed["expected_sessions"]]
low_attendance

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S002,gamma,3,6,3.0
S003,beta,4,6,6.0
S004,alpha,4,6,4.0
S005,gamma,5,6,6.0
S006,gamma,3,6,3.0
S007,gamma,5,6,5.0
S008,alpha,1,6,1.0
S009,alpha,1,6,1.0
S010,gamma,4,6,5.0


In [13]:
grouped=attendance_indexed.groupby("cohort")["attended_sessions"].mean()
grouped

cohort
alpha    3.875000
beta     2.600000
gamma    3.545455
Name: attended_sessions, dtype: float64

In [14]:
cohort_names = set(grouped.index.tolist())
cohort_names==cleaned #verify cohort names match to cleaned cohorts

True

Task5

In [15]:
attendance_indexed["attendance_ok"]=attendance_indexed["attended_sessions"]>=attendance_indexed["expected_sessions"]
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance,attendance_ok
student_id,,,,,
S001,alpha,6,6,7.0,True
S002,gamma,3,6,3.0,False
S003,beta,4,6,6.0,False
S004,alpha,4,6,4.0,False
S005,gamma,5,6,6.0,False
S006,gamma,3,6,3.0,False
S007,gamma,5,6,5.0,False
S008,alpha,1,6,1.0,False
S009,alpha,1,6,1.0,False


In [16]:
attendance_indexed["attendance_ok"] = attendance_indexed["attended_sessions"] >= attendance_indexed["expected_sessions"]
low_attendance = attendance_indexed[attendance_indexed["attended_sessions"] < attendance_indexed["expected_sessions"]]
all_false = (low_attendance["attendance_ok"] == False).all()
print(all_false)

True
